# MongoDB Geospatial Query Operators

In [1]:
from pymongo import MongoClient
from pathlib import Path

In [ ]:
MONGO_DB_CONNECTION_STRING = "mongodb://localhost:27017/"

# Creates database called "geo_example"
db = MongoClient(MONGO_DB_CONNECTION_STRING).geo_example

In [ ]:
# Creates container called "places" with defined index
db.places.create_index([
    ('location', '2dsphere')])

db.places.insert_one({
    'Entity Name': 'Walmart',
    'location': {
        'type': 'Point',
        'coordinates': [
            -100.98817410082268,
            10.72788705499784
        ],
    }
})

db.places.insert_one({
    'Entity Name': 'Target',
    'location': {
        'type': 'Polygon',
        'coordinates': [[[ 17.60083012593064, 78.18557739257812], 
                            [ 17.16834652544664, 78.19381713867188], 
                            [ 17.17490690610013, 78.739013671875], 
                            [ 17.613919673106714, 78.73489379882812],
                            [ 17.60083012593064, 78.18557739257812]
                          ]]
    }
})

result = db.command(
    'geoNear', 'places',
    near={
        'type': 'Point',
        'coordinates': [
            -73.991084,
            40.735863]},
    spherical=True,
    num=10)

print("geoNear results")
for x in result["results"]:
    print(x)


db.places.drop() 

In [ ]:
db.places.create_index([
    ('loc', '2dsphere')])

poly1 = { "type" : "Polygon", "coordinates" : [[[0, 0], [3, 0], [0, 3], [0, 0]]] }
poly2 = { "type" : "Polygon", "coordinates" : [[[1, 1], [2, 1], [1, 2], [1, 1]]] }

# poly1 is a similar triangle inside poly2
poly3 = { "type" : "Polygon", "coordinates" : [[[1, 0], [-2, 0], [1, 3], [1, 0]]] }

# poly3 is poly1 flipped around its "vertical" edge, then bumped over one unit, so it intersects poly1 but is not contained in it
db.places.insert_one({ "_id":1, "loc" : poly2 })
db.places.insert_one({ "_id":2, "loc" : poly3 })

result1 = db.places.find({ "loc" : {
    "$geoIntersects" : {
        "$geometry" : poly1
    }
} })

print("geoIntersects results")
for x in result1:
    print(x)


result2 = db.places.find({ "loc" : {
    "$geoWithin" : {
        "$geometry" : poly1
    }
} })

print("geoWithin results")
for x in result2:
    print(x)

db.places.drop() 